# importing libraries

In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [18]:
print(tf.__version__)

2.0.0


I will use the Fashion-MNIST dataset, so the goal is to perform a multiclass classification of images. First, I have to load the training and test dataset. Fashion-MNIST is available as one of the Keras built-in datasets

In [19]:
fashion_mnist=keras.datasets.fashion_mnist

In [41]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

The images have been already preprocessed, so currently, the dataset contains one channel (gray-scale) of color values in the range 0-255). I want to scale the values to range between 0 and 1, so I divide them by 255.

In [42]:
train_images=train_images/255.0
test_images=test_images/255.0

In [43]:
train_images[0].shape

(28, 28)

In [45]:
test_images[0].shape

(28, 28, 1)

I am going to reshape the dataset to use it as an input of the convolutional layer.
reshape this (28,28) into how many number of images we have,what is the pixel along with the 4 dimensions 

In [46]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

# Parameters

Keras-tuner needs a function that accepts the set of parameters and returns a compiled model, so I have to define such function.

There are four kinds of parameters available: range, choice, linear, and fixed.

# Range

The range returns integer values between the given minimum and maximum. The values are incremented by the step parameter.

In [61]:
# hp.Range('conv_1_filter', min_value=64, max_value=128, step=16)


# Linear

The liner parameter is similar to the range but works with float numbers. In this case, the step is called resolution.

In [62]:
# hp.Linear('learning_rate', min_value=0.01, max_value=0.1, resolution=0.1)


# Choice
The choice parameter is much simpler. We give it a list of values, and it returns one of them.

In [63]:
#hp.Choice('learning_rate', values=[1e-2, 1e-3])

# Fixed
Finally, we can set a constant as the parameter value. It is useful when we want to let keras-tuner tune all parameters except one. The fixed parameter works only with the predefined models: Xception and ResNet.

In [64]:
#hp.Fixed('learning_rate', value=1e-4)

# How to define the model
Here is my function that builds a neural network using the parameters given by keras-tuner. I will parameterize all layers and the learning rate, to show that it is possible.

In [47]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        # Range of integer with conv_1_filter the minimum 32 and max value 128
        filters=hp.Range('conv_1_filter', min_value=32, max_value=128, step=16),
        
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Range('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Range('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [50]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Configure the tuner

When the function is ready, I have to configure the tuner. We need to specify the objective, which is the metric used to compare models. In this case, I want to use validation set accuracy.

The other important parameter is the number of trails. That parameter tells the tuner how many hyperparameter combinations it has to test.

I must also specify the name and the output directory. It tells the tuner where it should store the debugging data.

In [51]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="mnist fashion")

# Hyperparameter tuning
Now, I have a configured tuner. It is time to run it. I need the training datasets, and the number of epochs is every trial. I must also specify the validation dataset or the percentage of training dataset that will be used for validation.

In [54]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3


 5824/54000 [==>...........................] - ETA: 54:10 - loss: 2.3050 - accuracy: 0.218 - ETA: 32:24 - loss: 2.3304 - accuracy: 0.140 - ETA: 25:05 - loss: 2.2751 - accuracy: 0.166 - ETA: 21:17 - loss: 2.2670 - accuracy: 0.195 - ETA: 19:10 - loss: 2.2175 - accuracy: 0.237 - ETA: 17:34 - loss: 2.1821 - accuracy: 0.244 - ETA: 16:23 - loss: 2.1112 - accuracy: 0.294 - ETA: 15:34 - loss: 2.0336 - accuracy: 0.312 - ETA: 14:50 - loss: 1.9654 - accuracy: 0.336 - ETA: 14:16 - loss: 1.9059 - accuracy: 0.350 - ETA: 13:53 - loss: 1.8257 - accuracy: 0.372 - ETA: 13:32 - loss: 1.7509 - accuracy: 0.393 - ETA: 13:12 - loss: 1.7863 - accuracy: 0.396 - ETA: 12:55 - loss: 1.7373 - accuracy: 0.406 - ETA: 12:38 - loss: 1.6886 - accuracy: 0.422 - ETA: 12:23 - loss: 1.6502 - accuracy: 0.443 - ETA: 12:12 - loss: 1.6219 - accuracy: 0.455 - ETA: 12:00 - loss: 1.6199 - accuracy: 0.456 - ETA: 11:49 - loss: 1.6034 - accuracy: 0.468 - ETA: 11:40 - loss: 1.5605 - accuracy: 0.485 - ETA: 11:33 - loss: 1.5399 - accur

17728/54000 [========>.....................] - ETA: 7:17 - loss: 0.6004 - accuracy: 0.78 - ETA: 7:17 - loss: 0.5995 - accuracy: 0.78 - ETA: 7:16 - loss: 0.5989 - accuracy: 0.78 - ETA: 7:16 - loss: 0.5986 - accuracy: 0.78 - ETA: 7:15 - loss: 0.5977 - accuracy: 0.79 - ETA: 7:15 - loss: 0.5975 - accuracy: 0.79 - ETA: 7:14 - loss: 0.5966 - accuracy: 0.79 - ETA: 7:14 - loss: 0.5959 - accuracy: 0.79 - ETA: 7:13 - loss: 0.5955 - accuracy: 0.79 - ETA: 7:13 - loss: 0.5947 - accuracy: 0.79 - ETA: 7:12 - loss: 0.5946 - accuracy: 0.79 - ETA: 7:12 - loss: 0.5937 - accuracy: 0.79 - ETA: 7:12 - loss: 0.5936 - accuracy: 0.79 - ETA: 7:11 - loss: 0.5924 - accuracy: 0.79 - ETA: 7:11 - loss: 0.5917 - accuracy: 0.79 - ETA: 7:11 - loss: 0.5921 - accuracy: 0.79 - ETA: 7:10 - loss: 0.5909 - accuracy: 0.79 - ETA: 7:10 - loss: 0.5902 - accuracy: 0.79 - ETA: 7:09 - loss: 0.5894 - accuracy: 0.79 - ETA: 7:09 - loss: 0.5893 - accuracy: 0.79 - ETA: 7:08 - loss: 0.5892 - accuracy: 0.79 - ETA: 7:08 - loss: 0.5883 - ac

23680/54000 [============>.................] - ETA: 5:58 - loss: 0.5322 - accuracy: 0.81 - ETA: 5:58 - loss: 0.5320 - accuracy: 0.81 - ETA: 5:57 - loss: 0.5318 - accuracy: 0.81 - ETA: 5:57 - loss: 0.5318 - accuracy: 0.81 - ETA: 5:57 - loss: 0.5314 - accuracy: 0.81 - ETA: 5:56 - loss: 0.5313 - accuracy: 0.81 - ETA: 5:56 - loss: 0.5313 - accuracy: 0.81 - ETA: 5:55 - loss: 0.5308 - accuracy: 0.81 - ETA: 5:55 - loss: 0.5304 - accuracy: 0.81 - ETA: 5:55 - loss: 0.5300 - accuracy: 0.81 - ETA: 5:54 - loss: 0.5302 - accuracy: 0.81 - ETA: 5:54 - loss: 0.5296 - accuracy: 0.81 - ETA: 5:53 - loss: 0.5293 - accuracy: 0.81 - ETA: 5:53 - loss: 0.5287 - accuracy: 0.81 - ETA: 5:53 - loss: 0.5288 - accuracy: 0.81 - ETA: 5:52 - loss: 0.5285 - accuracy: 0.81 - ETA: 5:52 - loss: 0.5282 - accuracy: 0.81 - ETA: 5:51 - loss: 0.5277 - accuracy: 0.81 - ETA: 5:51 - loss: 0.5272 - accuracy: 0.81 - ETA: 5:51 - loss: 0.5274 - accuracy: 0.81 - ETA: 5:50 - loss: 0.5271 - accuracy: 0.81 - ETA: 5:50 - loss: 0.5271 - ac

29632/54000 [===============>..............] - ETA: 4:54 - loss: 0.4913 - accuracy: 0.82 - ETA: 4:54 - loss: 0.4910 - accuracy: 0.82 - ETA: 4:53 - loss: 0.4909 - accuracy: 0.82 - ETA: 4:53 - loss: 0.4907 - accuracy: 0.82 - ETA: 4:53 - loss: 0.4906 - accuracy: 0.82 - ETA: 4:52 - loss: 0.4905 - accuracy: 0.82 - ETA: 4:52 - loss: 0.4909 - accuracy: 0.82 - ETA: 4:52 - loss: 0.4907 - accuracy: 0.82 - ETA: 4:51 - loss: 0.4908 - accuracy: 0.82 - ETA: 4:51 - loss: 0.4906 - accuracy: 0.82 - ETA: 4:50 - loss: 0.4906 - accuracy: 0.82 - ETA: 4:50 - loss: 0.4902 - accuracy: 0.82 - ETA: 4:50 - loss: 0.4901 - accuracy: 0.82 - ETA: 4:49 - loss: 0.4900 - accuracy: 0.82 - ETA: 4:49 - loss: 0.4898 - accuracy: 0.82 - ETA: 4:49 - loss: 0.4894 - accuracy: 0.82 - ETA: 4:48 - loss: 0.4894 - accuracy: 0.82 - ETA: 4:48 - loss: 0.4890 - accuracy: 0.82 - ETA: 4:48 - loss: 0.4889 - accuracy: 0.82 - ETA: 4:47 - loss: 0.4888 - accuracy: 0.82 - ETA: 4:47 - loss: 0.4884 - accuracy: 0.82 - ETA: 4:47 - loss: 0.4880 - ac

...

35712/54000 [==================>...........] - ETA: 1:38 - loss: 0.1963 - accuracy: 0.92 - ETA: 1:38 - loss: 0.1964 - accuracy: 0.92 - ETA: 1:38 - loss: 0.1964 - accuracy: 0.92 - ETA: 1:38 - loss: 0.1964 - accuracy: 0.92 - ETA: 1:38 - loss: 0.1965 - accuracy: 0.92 - ETA: 1:38 - loss: 0.1964 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1963 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1963 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1962 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1963 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1961 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1961 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1960 - accuracy: 0.92 - ETA: 1:37 - loss: 0.1958 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1957 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1957 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1959 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1958 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1957 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1958 - accuracy: 0.92 - ETA: 1:36 - loss: 0.1958 - accuracy: 0.92 - ETA: 1:35 - loss: 0.1957 - ac

41664/54000 [======================>.......] - ETA: 1:14 - loss: 0.1947 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1946 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1946 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1946 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1946 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1947 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1946 - accuracy: 0.92 - ETA: 1:13 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1949 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1949 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1950 - accuracy: 0.92 - ETA: 1:12 - loss: 0.1951 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1950 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1950 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1949 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1948 - accuracy: 0.92 - ETA: 1:11 - loss: 0.1948 - ac

47616/54000 [=========================>....] - ETA: 49s - loss: 0.1962 - accuracy: 0.926 - ETA: 49s - loss: 0.1964 - accuracy: 0.926 - ETA: 49s - loss: 0.1964 - accuracy: 0.926 - ETA: 49s - loss: 0.1963 - accuracy: 0.926 - ETA: 49s - loss: 0.1965 - accuracy: 0.926 - ETA: 49s - loss: 0.1964 - accuracy: 0.926 - ETA: 48s - loss: 0.1963 - accuracy: 0.926 - ETA: 48s - loss: 0.1964 - accuracy: 0.926 - ETA: 48s - loss: 0.1965 - accuracy: 0.926 - ETA: 48s - loss: 0.1965 - accuracy: 0.926 - ETA: 48s - loss: 0.1965 - accuracy: 0.926 - ETA: 48s - loss: 0.1965 - accuracy: 0.926 - ETA: 48s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1964 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1965 - accuracy: 0.926 - ETA: 47s - loss: 0.1967 - accuracy: 0.926 - ETA: 46s - loss: 0.1968 - acc

53664/54000 [============================>.] - ETA: 25s - loss: 0.1965 - accuracy: 0.926 - ETA: 25s - loss: 0.1965 - accuracy: 0.926 - ETA: 25s - loss: 0.1964 - accuracy: 0.926 - ETA: 25s - loss: 0.1964 - accuracy: 0.926 - ETA: 24s - loss: 0.1965 - accuracy: 0.926 - ETA: 24s - loss: 0.1965 - accuracy: 0.926 - ETA: 24s - loss: 0.1965 - accuracy: 0.926 - ETA: 24s - loss: 0.1965 - accuracy: 0.926 - ETA: 24s - loss: 0.1964 - accuracy: 0.926 - ETA: 24s - loss: 0.1964 - accuracy: 0.926 - ETA: 24s - loss: 0.1964 - accuracy: 0.926 - ETA: 24s - loss: 0.1964 - accuracy: 0.926 - ETA: 23s - loss: 0.1963 - accuracy: 0.926 - ETA: 23s - loss: 0.1963 - accuracy: 0.926 - ETA: 23s - loss: 0.1963 - accuracy: 0.926 - ETA: 23s - loss: 0.1962 - accuracy: 0.926 - ETA: 23s - loss: 0.1963 - accuracy: 0.926 - ETA: 23s - loss: 0.1965 - accuracy: 0.926 - ETA: 23s - loss: 0.1965 - accuracy: 0.926 - ETA: 23s - loss: 0.1966 - accuracy: 0.926 - ETA: 22s - loss: 0.1967 - accuracy: 0.926 - ETA: 22s - loss: 0.1967 - acc

54000/54000 [==============================] - ETA: 1s - loss: 0.1975 - accuracy: 0.92 - ETA: 1s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - ETA: 0s - loss: 0.1974 - accuracy: 0.92 - ETA: 0s - loss: 0.1975 - accuracy: 0.92 - 220s 4ms/sample - loss: 0.1976 - accuracy: 0.9256 - val_loss: 0.2570 - val_accuracy: 0.9083


# Using the model
When the search is done, I can get the best model and either start using it or continue training.

In [55]:
model=tuner_search.get_best_models(num_models=1)[0]

In [58]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,639,914
Trainable params: 2,639,914
Non-trainable params: 0
_________________________________________________________________


In this example, I trained the model for only two epochs, so I will continue training it, starting from the third epoch.

In [59]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10


 5888/54000 [==>...........................] - ETA: 14:23 - loss: 0.1125 - accuracy: 0.968 - ETA: 8:59 - loss: 0.1218 - accuracy: 0.968 - ETA: 7:09 - loss: 0.1886 - accuracy: 0.94 - ETA: 6:19 - loss: 0.1767 - accuracy: 0.94 - ETA: 5:47 - loss: 0.1506 - accuracy: 0.95 - ETA: 5:24 - loss: 0.1527 - accuracy: 0.94 - ETA: 5:12 - loss: 0.1411 - accuracy: 0.94 - ETA: 4:59 - loss: 0.1366 - accuracy: 0.94 - ETA: 4:50 - loss: 0.1316 - accuracy: 0.94 - ETA: 4:43 - loss: 0.1267 - accuracy: 0.95 - ETA: 4:37 - loss: 0.1336 - accuracy: 0.94 - ETA: 4:33 - loss: 0.1339 - accuracy: 0.94 - ETA: 4:31 - loss: 0.1328 - accuracy: 0.94 - ETA: 4:30 - loss: 0.1494 - accuracy: 0.93 - ETA: 4:27 - loss: 0.1435 - accuracy: 0.94 - ETA: 4:23 - loss: 0.1378 - accuracy: 0.94 - ETA: 4:22 - loss: 0.1398 - accuracy: 0.94 - ETA: 4:21 - loss: 0.1407 - accuracy: 0.94 - ETA: 4:20 - loss: 0.1361 - accuracy: 0.94 - ETA: 4:18 - loss: 0.1354 - accuracy: 0.94 - ETA: 4:16 - loss: 0.1362 - accuracy: 0.94 - ETA: 4:14 - loss: 0.1362 -

17792/54000 [========>.....................] - ETA: 3:05 - loss: 0.1344 - accuracy: 0.95 - ETA: 3:05 - loss: 0.1344 - accuracy: 0.95 - ETA: 3:04 - loss: 0.1350 - accuracy: 0.95 - ETA: 3:04 - loss: 0.1351 - accuracy: 0.95 - ETA: 3:04 - loss: 0.1353 - accuracy: 0.95 - ETA: 3:04 - loss: 0.1360 - accuracy: 0.95 - ETA: 3:04 - loss: 0.1360 - accuracy: 0.95 - ETA: 3:03 - loss: 0.1361 - accuracy: 0.95 - ETA: 3:03 - loss: 0.1361 - accuracy: 0.95 - ETA: 3:03 - loss: 0.1370 - accuracy: 0.95 - ETA: 3:03 - loss: 0.1370 - accuracy: 0.94 - ETA: 3:03 - loss: 0.1368 - accuracy: 0.95 - ETA: 3:02 - loss: 0.1368 - accuracy: 0.94 - ETA: 3:02 - loss: 0.1368 - accuracy: 0.94 - ETA: 3:02 - loss: 0.1371 - accuracy: 0.94 - ETA: 3:02 - loss: 0.1371 - accuracy: 0.94 - ETA: 3:02 - loss: 0.1368 - accuracy: 0.94 - ETA: 3:02 - loss: 0.1371 - accuracy: 0.94 - ETA: 3:01 - loss: 0.1370 - accuracy: 0.94 - ETA: 3:01 - loss: 0.1369 - accuracy: 0.94 - ETA: 3:01 - loss: 0.1367 - accuracy: 0.95 - ETA: 3:01 - loss: 0.1366 - ac

23744/54000 [============>.................] - ETA: 2:35 - loss: 0.1404 - accuracy: 0.94 - ETA: 2:35 - loss: 0.1407 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1408 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1409 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1410 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1408 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1407 - accuracy: 0.94 - ETA: 2:34 - loss: 0.1409 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1411 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1411 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1411 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1411 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1410 - accuracy: 0.94 - ETA: 2:33 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1414 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:32 - loss: 0.1413 - accuracy: 0.94 - ETA: 2:31 - loss: 0.1412 - accuracy: 0.94 - ETA: 2:31 - loss: 0.1418 - ac

29696/54000 [===============>..............] - ETA: 2:05 - loss: 0.1425 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1425 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1426 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1426 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1427 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1426 - accuracy: 0.94 - ETA: 2:05 - loss: 0.1425 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1424 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1425 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1425 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1423 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1423 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1422 - accuracy: 0.94 - ETA: 2:04 - loss: 0.1421 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1419 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1421 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1420 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1419 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1419 - accuracy: 0.94 - ETA: 2:03 - loss: 0.1420 - accuracy: 0.94 - ETA: 2:02 - loss: 0.1419 - accuracy: 0.94 - ETA: 2:02 - loss: 0.1418 - ac

...

35712/54000 [==================>...........] - ETA: 1:18 - loss: 0.0264 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0264 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0263 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0265 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0265 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0267 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0267 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0267 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0267 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0266 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0268 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0268 - ac

41664/54000 [======================>.......] - ETA: 58s - loss: 0.0272 - accuracy: 0.990 - ETA: 58s - loss: 0.0272 - accuracy: 0.990 - ETA: 58s - loss: 0.0272 - accuracy: 0.990 - ETA: 58s - loss: 0.0271 - accuracy: 0.990 - ETA: 58s - loss: 0.0271 - accuracy: 0.990 - ETA: 58s - loss: 0.0272 - accuracy: 0.990 - ETA: 58s - loss: 0.0272 - accuracy: 0.990 - ETA: 58s - loss: 0.0274 - accuracy: 0.990 - ETA: 58s - loss: 0.0274 - accuracy: 0.990 - ETA: 57s - loss: 0.0274 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 57s - loss: 0.0273 - accuracy: 0.990 - ETA: 56s - loss: 0.0272 - accuracy: 0.990 - ETA: 56s - loss: 0.0272 - accuracy: 0.990 - ETA: 56s - loss: 0.0273 - acc

47616/54000 [=========================>....] - ETA: 39s - loss: 0.0286 - accuracy: 0.990 - ETA: 39s - loss: 0.0287 - accuracy: 0.990 - ETA: 39s - loss: 0.0288 - accuracy: 0.990 - ETA: 39s - loss: 0.0287 - accuracy: 0.990 - ETA: 39s - loss: 0.0287 - accuracy: 0.990 - ETA: 39s - loss: 0.0287 - accuracy: 0.990 - ETA: 39s - loss: 0.0287 - accuracy: 0.990 - ETA: 38s - loss: 0.0287 - accuracy: 0.990 - ETA: 38s - loss: 0.0287 - accuracy: 0.990 - ETA: 38s - loss: 0.0288 - accuracy: 0.989 - ETA: 38s - loss: 0.0288 - accuracy: 0.990 - ETA: 38s - loss: 0.0288 - accuracy: 0.989 - ETA: 38s - loss: 0.0288 - accuracy: 0.989 - ETA: 38s - loss: 0.0288 - accuracy: 0.990 - ETA: 38s - loss: 0.0287 - accuracy: 0.990 - ETA: 38s - loss: 0.0287 - accuracy: 0.990 - ETA: 38s - loss: 0.0287 - accuracy: 0.990 - ETA: 37s - loss: 0.0287 - accuracy: 0.990 - ETA: 37s - loss: 0.0287 - accuracy: 0.990 - ETA: 37s - loss: 0.0287 - accuracy: 0.990 - ETA: 37s - loss: 0.0286 - accuracy: 0.990 - ETA: 37s - loss: 0.0287 - acc

53696/54000 [============================>.] - ETA: 20s - loss: 0.0307 - accuracy: 0.989 - ETA: 20s - loss: 0.0307 - accuracy: 0.989 - ETA: 20s - loss: 0.0307 - accuracy: 0.989 - ETA: 20s - loss: 0.0306 - accuracy: 0.989 - ETA: 20s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0307 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 19s - loss: 0.0306 - accuracy: 0.989 - ETA: 18s - loss: 0.0306 - accuracy: 0.989 - ETA: 18s - loss: 0.0305 - accuracy: 0.989 - ETA: 18s - loss: 0.0305 - accuracy: 0.989 - ETA: 18s - loss: 0.0305 - accuracy: 0.989 - ETA: 18s - loss: 0.0306 - accuracy: 0.989 - ETA: 18s - loss: 0.0306 - accuracy: 0.989 - ETA: 18s - loss: 0.0306 - acc

54000/54000 [==============================] - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0306 - accuracy: 0.98 - ETA: 0s - loss: 0.0305 - accuracy: 0.98 - 179s 3ms/sample - loss: 0.0306 - accuracy: 0.9894 - val_loss: 0.5006 - val_accuracy: 0.9127
